# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [15]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re
import pickle
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///Disaster_Messages.db')
df = pd.read_sql_table('Message_Responses',engine)
# df.head()
X =df['message'] 
y =df.iloc[:,4:] 



### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
       
    text=re.sub(r"[^a-zA-Z0-9]"," ",text)

    tokens = word_tokenize(text)
    stopwords1=stopwords.words("english")
    tokens = [w for w in tokens if w not in stopwords1]
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens   
    

In [5]:
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                     ('tfidf',TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=142)
fitted_pipe=pipeline.fit(X_train,y_train)
fitted_pipe




Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# def evaluation_metrics(pipeline,X_train,y_train,X_test,y_test):

# y_train_pred=fitted_pipe.predict(X_train) 
y_test_pred=fitted_pipe.predict(X_test)

for i in range(len(y.columns)):
    
    print(y_test.columns[i],'category metrics :')
    print(classification_report(y_test.iloc[:,i],y_test_pred[:,i]))
    


related category metrics :
             precision    recall  f1-score   support

          0       0.64      0.46      0.54      1867
          1       0.84      0.91      0.88      5941
          2       0.28      0.44      0.34        57

avg / total       0.79      0.80      0.79      7865

request category metrics :
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      6517
          1       0.78      0.45      0.57      1348

avg / total       0.88      0.88      0.87      7865

offer category metrics :
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      7820
          1       0.00      0.00      0.00        45

avg / total       0.99      0.99      0.99      7865

aid_related category metrics :
             precision    recall  f1-score   support

          0       0.76      0.84      0.80      4617
          1       0.73      0.63      0.67      3248

avg / total       0.75      0.7

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# fitted_pipe.get_params().keys()
fitted_pipe.get_params()

parameters = {'clf__estimator__n_estimators':[100,150,200],
              'clf__estimator__min_samples_split': [2,5,10]}
               

cv = GridSearchCV(pipeline,param_grid=parameters,n_jobs=-1,cv=3,verbose=2)

In [11]:
cv.fit(X_train,y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 5.9min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  6.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 5.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 5.9min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=150 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=150, total= 8.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=150 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=150, total= 8.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=150 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=150, total= 8.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200, total=11.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200

[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 211.8min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [100, 150, 200], 'clf__estimator__min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [12]:
print(cv.best_params_)
new_cv_model=cv.best_estimator_

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 150}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
y_test_pred=new_cv_model.predict(X_test)

for i in range(len(y.columns)):
    
    print(y_test.columns[i],'category metrics :')
    print(classification_report(y_test.iloc[:,i],y_test_pred[:,i]))

related category metrics :
             precision    recall  f1-score   support

          0       0.71      0.40      0.52      1867
          1       0.83      0.95      0.89      5941
          2       0.30      0.44      0.35        57

avg / total       0.80      0.81      0.79      7865

request category metrics :
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      6517
          1       0.82      0.50      0.62      1348

avg / total       0.89      0.90      0.88      7865

offer category metrics :
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      7820
          1       0.00      0.00      0.00        45

avg / total       0.99      0.99      0.99      7865

aid_related category metrics :
             precision    recall  f1-score   support

          0       0.80      0.83      0.82      4617
          1       0.75      0.71      0.72      3248

avg / total       0.78      0.7

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
pipeline_2 = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                     ('tfidf',TfidfTransformer()),
                     ('clf', MultiOutputClassifier(AdaBoostClassifier()))])



In [17]:
parameters = {'clf__estimator__n_estimators':[30,50],
              'clf__estimator__learning_rate': [0.01,0.03]}
cv_2 = GridSearchCV(pipeline_2,param_grid=parameters,n_jobs=4,cv=3,verbose=2)


In [18]:
cv_2.fit(X_train,y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=30 
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=30 
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=30 
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=30, total= 3.2min
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=30, total= 3.2min
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=30, total= 3.2min
[CV] clf__estimator__learning_rate=0.03, clf__estimator__n_estimators=30 
[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, total= 5.0min
[CV] clf__estimator__learning_rate=0.03, clf__estimator__n_estimat

[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 15.0min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 15.0min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'clf__estimator__n_estimators': [30, 50], 'clf__estimator__learning_rate': [0.01, 0.03]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [20]:
print(cv_2.best_params_)
new_cv_model_2=cv_2.best_estimator_

{'clf__estimator__learning_rate': 0.03, 'clf__estimator__n_estimators': 50}


In [21]:
y_test_pred_2=new_cv_model_2.predict(X_test)

for i in range(len(y.columns)):
    
    print(y_test.columns[i],'category metrics :')
    print(classification_report(y_test.iloc[:,i],y_test_pred_2[:,i]))

related category metrics :
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1867
          1       0.76      1.00      0.86      5941
          2       0.00      0.00      0.00        57

avg / total       0.57      0.76      0.65      7865

request category metrics :
             precision    recall  f1-score   support

          0       0.85      0.99      0.91      6517
          1       0.82      0.13      0.23      1348

avg / total       0.84      0.85      0.80      7865

offer category metrics :
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      7820
          1       0.00      0.00      0.00        45

avg / total       0.99      0.99      0.99      7865

aid_related category metrics :
             precision    recall  f1-score   support

          0       0.64      0.96      0.77      4617
          1       0.80      0.23      0.36      3248

avg / total       0.71      0.6

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [ ]:
file=pickle.dumps('model.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.